In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from src.utilsgru import crear_ventanas_temporales, train_model, train_model_earlystop, crear_features_con_lags, train_mlp_model
from models.gru import GRUModel, GRUDataset
from torch.utils.data import DataLoader

In [5]:
df = pd.read_csv("data/modelado/ds_modelado.csv")
df_verano = df[df["estacion_del_anio"] == 1].copy()

In [6]:
target =  "N_arribos_intervalo"
cols_a_excluir = [
    "N_arribos_intervalo",
    "id_recorrido",
    "id_usuario",
    "id_estacion_destino",  # borre cosas de destino porq siento q son leakage onda el modelo puede adivinar
    "barrio_destino",
    "zona_destino_cluster",
    "cantidad_estaciones_cercanas_destino",
    "año_destino", "mes_destino", "dia_destino",
    "hora_destino", "minuto_destino", "segundo_destino",
    "duracion_recorrido"
]
features = [col for col in df_verano.columns if col not in cols_a_excluir]

In [7]:
df_train, df_val = train_test_split(
    df_verano,
    test_size=0.2,
    random_state=42,
    stratify=df_verano["año_intervalo"]
)

In [8]:
# Escalar features X
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(df_train[features])
X_val_scaled = scaler_X.transform(df_val[features])


In [9]:
# Target sin escalar todavía
y_train = df_train[target].values
y_val = df_val[target].values

GRU (no funciona bien pero bueno I tried)

In [10]:
ventana = 6
X_train_seq, y_train_seq = crear_ventanas_temporales(X_train_scaled, y_train, ventana)
X_val_seq, y_val_seq = crear_ventanas_temporales(X_val_scaled, y_val, ventana)

In [11]:
print(f"X_train_seq shape: {X_train_seq.shape}")
print(f"y_train_seq shape: {y_train_seq.shape}")
print(f"Primeros valores y_train_seq: {y_train_seq[:5]}")

X_train_seq shape: (3053484, 6, 27)
y_train_seq shape: (3053484,)
Primeros valores y_train_seq: [3 1 3 1 1]


In [12]:
# Dataset y DataLoader con target escalado
train_dataset = GRUDataset(X_train_seq, y_train_seq )
val_dataset = GRUDataset(X_val_seq, y_val_seq)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=0, pin_memory=True)


In [13]:
# Instanciar modelo
model = GRUModel(input_dim=X_train_seq.shape[2], hidden_dim=128, num_layers=3)

print(len(train_dataset))
print(X_train_seq.shape)


3053484
(3053484, 6, 27)


In [14]:
# Finalmente entrenamos con la función que soporta el desescalado
#train_model(model, train_loader, val_loader, epochs=5, lr=0.001, scaler_y=scaler_y)

In [15]:
#train_model_earlystop(model, train_loader, val_loader, epochs=5, lr=0.001)

In [16]:
X_train_lag, y_train_lag = crear_features_con_lags(X_train_scaled, y_train, ventana)
X_val_lag, y_val_lag = crear_features_con_lags(X_val_scaled, y_val, ventana)
print("Shape X_train_lag:", X_train_lag.shape)


Shape X_train_lag: (3053484, 162)


In [17]:
# Escalar el target de las ventanas con lag
scaler_y = StandardScaler()
y_train_lag_scaled = scaler_y.fit_transform(y_train_lag.reshape(-1, 1)).flatten()
y_val_lag_scaled = scaler_y.transform(y_val_lag.reshape(-1, 1)).flatten()

In [18]:
#model = train_mlp_model(X_train_lag, y_train_lag_scaled, X_val_lag, y_val_lag_scaled, input_dim=X_train_lag.shape[1],scaler_y=scaler_y)


In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(
    n_estimators=20,     # menos árboles para entrenar rápido
    max_depth=15,        # profundidad limitada
    random_state=42,
    n_jobs=-1            # usa todos los cores disponibles
)
rf.fit(X_train_lag[:500000], y_train_lag[:500000])  # tomar 500k filas para prototipar

y_pred = rf.predict(X_val_lag)

print("MAE:", mean_absolute_error(y_val_lag, y_pred))
print("R2:", r2_score(y_val_lag, y_pred))

MAE: 1.4578143795824337
R2: -0.005774668254849402
